In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
%cd /content/drive/MyDrive/Laboratorio MLT/Project

/content/drive/MyDrive/Laboratorio MLT/Project


# Using model on cloud

In [10]:
!pip install transformers -q

In [85]:
from transformers import pipeline
qa_pipeline = pipeline(
    "question-answering",
    model="mrm8488/bert-multi-cased-finetuned-xquadv1",
    tokenizer="mrm8488/bert-multi-cased-finetuned-xquadv1"
)

def predict(context, question):
  answer = qa_pipeline({
    'context': context,
    'question': question})
  return answer

# Exporting model

In [77]:
from pathlib import Path
import transformers
from transformers.onnx import FeaturesManager
from transformers import AutoConfig, AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoModelForQuestionAnswering
from transformers import AutoModel

# load model and tokenizer
tokenizer_id="mrm8488/bert-multi-cased-finetuned-xquadv1"
model_id="mrm8488/bert-multi-cased-finetuned-xquadv1",

feature = "question-answering"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
model = AutoModelForQuestionAnswering.from_pretrained(tokenizer_id)

model = BertModel.from_pretrained("mrm8488/bert-multi-cased-finetuned-xquadv1")

# load config
model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(model, feature=feature)
onnx_config = model_onnx_config(model.config)

# export
"""
onnx_inputs, onnx_outputs = transformers.onnx.export(
        model = model,
        config=onnx_config,
        opset=13,
        output=Path("trfs-model.onnx")
)"""
model.save_pretrained("ltrfs-model.onnx")

Some weights of the model checkpoint at mrm8488/bert-multi-cased-finetuned-xquadv1 were not used when initializing BertModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
!pip install onnxruntime -q

In [49]:
import onnx
import onnxruntime as rt

In [50]:
sess = rt.InferenceSession("trfs-model.onnx", providers=rt.get_available_providers())

print("input name='{}' and shape={}".format(sess.get_inputs()[0].name, sess.get_inputs()[0].shape))
print("output name='{}' and shape={}".format(sess.get_outputs()[0].name, sess.get_outputs()[0].shape))

input name='input_ids' and shape=['batch', 'sequence']
output name='start_logits' and shape=['batch', 'sequence']


In [80]:
text="Manuel Romero has been working hardly in the repository hugginface/transformers lately",
question= "Who has been working hard for hugginface/transformers lately?"

In [84]:
tokenizer_id="mrm8488/bert-multi-cased-finetuned-xquadv1"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)

onnx_model = onnx.load("trfs-model.onnx")
inputs = tokenizer(question, question, add_special_tokens=True, return_tensors='np')
# outputs = onnx_model.run(input_feed=dict(tokenizer(question, text, add_special_tokens=True, return_tensors='np')), output_names=None)
outputs = onnx_model.run(input_feed=dict(inputs), output_names=None)

AttributeError: ignored